Check Create_Own_pkl_file.ipynb file to get the pkl file used here.

In [ ]:
import pandas as pd
import numpy as np
import pickle

def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

games = reduce_memory(pd.read_csv('./final_games_data_forDesc.csv'))
n_recommendation = 20

similarity=pickle.load(open("./similarity_forDesc.pkl",'rb'))

def recommend(game):
    index = games[games['title'] == game].index[0]
    sim_scores = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    game_lists=[]
    for i in sim_scores[1:n_recommendation]:
        game_lists.append(games.iloc[i[0]].title)
    return game_lists

recommend('Call of Duty®: Black Ops Cold War')

In [ ]:
recommend("Animal Academy")